In [8]:
import pandas as pd
import re
import numpy as np
import os

In [9]:
# pull data, sort out paths

proj_dir = os.getcwd()
data_dir = os.path.join(proj_dir, 'data' )
os.chdir( proj_dir )
os.system("rm -rf {}".format( data_dir ))
os.system('mkdir data')
os.chdir( data_dir )
os.system("kaggle competitions download -c march-machine-learning-mania-2023")
os.system( 'unzip {}.zip'.format( os.path.join(data_dir, 'march-machine-learning-mania-2023')) )

100%|██████████| 28.5M/28.5M [00:02<00:00, 13.0MB/s]



Archive:  /Users/nickbachelder/Desktop/Kaggle/Madness/data/march-machine-learning-mania-2023.zip
  inflating: Cities.csv              
  inflating: Conferences.csv         
  inflating: MConferenceTourneyGames.csv  
  inflating: MGameCities.csv         
  inflating: MMasseyOrdinals_thru_Season2023_Day128.csv  
  inflating: MNCAATourneyCompactResults.csv  
  inflating: MNCAATourneyDetailedResults.csv  
  inflating: MNCAATourneySeedRoundSlots.csv  
  inflating: MNCAATourneySeeds.csv   
  inflating: MNCAATourneySlots.csv   
  inflating: MRegularSeasonCompactResults.csv  
  inflating: MRegularSeasonDetailedResults.csv  
  inflating: MSeasons.csv            
  inflating: MSecondaryTourneyCompactResults.csv  
  inflating: MSecondaryTourneyTeams.csv  
  inflating: MTeamCoaches.csv        
  inflating: MTeamConferences.csv    
  inflating: MTeamSpellings.csv      
  inflating: MTeams.csv              
  inflating: SampleSubmission2023.csv  
  inflating: WGameCities.csv         
  inflating: W

0

In [53]:
## Get tourney seed data

mseed = pd.read_csv("MNCAATourneySeeds.csv")
mseed['M_W'] = 'M'
wseed = pd.read_csv("WNCAATourneySeeds.csv")
wseed['M_W'] = 'W'
seed = pd.concat([mseed, wseed])
seed['ReigonSeed'] = seed.Seed
seed['Seed'] = [seeds[1:] for seeds in seed.Seed]
seed['play_in_seed'] = list(map(lambda x : x[-1].isalpha(), seed.Seed))
seed['Seed'] = list(map(lambda x : int(re.sub('\D', "", x)), seed.Seed))
seed.to_csv('tourney_seeds.csv', index=False) 

mslot = pd.read_csv("MNCAATourneySlots.csv")
mslot['M_W'] = 'M'
wslot = pd.read_csv("WNCAATourneySlots.csv")
wslot['M_W'] = 'W'
slot = pd.concat([mslot, wslot])
conditions = [
    ["R1" in slot for slot in slot.Slot],
    ["R2" in slot for slot in slot.Slot],
    ["R3" in slot for slot in slot.Slot],
    ["R4" in slot for slot in slot.Slot],
    ["R5" in slot for slot in slot.Slot],
    ["R6" in slot for slot in slot.Slot],
]
choices = [1,2,3,4,5,6]
slot["GameRound"] = np.select(conditions, choices, default = 0)
slot
slot.to_csv('tourney_slots.csv', index=False) 


In [19]:
m_season_details = pd.read_csv("MRegularSeasonDetailedResults.csv")
m_season_details['M_W'] = 'M'
w_season_details = pd.read_csv("WRegularSeasonDetailedResults.csv")
m_season_details['M_W'] = 'W'
season_details = pd.concat([m_season_details, w_season_details])
season_details


winner_season_details = season_details[['M_W', "Season", "DayNum", "WTeamID", "LTeamID", "WScore", "LScore", "WLoc", "WFGM", 
'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR',
       'WAst', 'WTO', 'WStl', 'WBlk', 'WPF'
]].set_axis(['M_W', "Season", "DayNum", "TeamID", "OppTeamID", "Score", "OppScore", "Loc", "FGM", 
'FGA', 'FGM3', 'FGA3', 'FTM', 'FTA', 'OR', 'DR',
       'Ast', 'TO', 'Stl', 'Blk', 'PF'], axis=1, inplace=False)
winner_season_details['Result'] = 1
       
loser_season_details = season_details[['M_W', "Season", "DayNum", "WTeamID", "LTeamID", "WScore", "LScore", 
"WLoc", 'LFGM', 'LFGA', 'LFGM3', 'LFGA3',
       'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF'
]].set_axis(['M_W', "Season", "DayNum", "OppTeamID", "TeamID", "OppScore", "Score", "Loc", "FGM", 
'FGA', 'FGM3', 'FGA3', 'FTM', 'FTA', 'OR', 'DR',
       'Ast', 'TO', 'Stl', 'Blk', 'PF'], axis=1, inplace=False)
conditions = [(loser_season_details.Loc == "H"), 
(loser_season_details.Loc == "A"), 
(loser_season_details.Loc == "N")]
choices = ["A", "H", "N"]
loser_season_details['Result'] = 0
loser_season_details["Loc"] = np.select(conditions, choices)
season_details = pd.concat([winner_season_details, loser_season_details])
season_details.to_csv('game_results.csv', index=False) 

In [22]:
season_rankings_m = pd.read_csv("MMasseyOrdinals_thru_Season2023_Day128.csv")
season_rankings_m.to_csv("season_rankings_men.csv", index = False)

In [24]:
m_tourney_results = pd.read_csv("MNCAATourneyCompactResults.csv")
m_tourney_results['M_W'] = 'M'
w_tourney_results = pd.read_csv("WNCAATourneyCompactResults.csv")
w_tourney_results['M_W'] = 'W'
tourney_details = pd.concat([m_tourney_results, w_tourney_results])
tourney_details

tourney_details.to_csv('tourney_results.csv', index=False) 

In [42]:
mteams = pd.read_csv("MTeams.csv")
mteams['M_W'] = 'M'
wteams = pd.read_csv("WTeams.csv")
wteams['M_W'] = 'W'
teams = pd.concat([mteams, wteams])
teams.to_csv('teams.csv', index=False) 